In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

from scipy import optimize

import tensorflow as tf
import torch
import os
os.chdir('..')

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from IPython.display import clear_output
import datetime
import matplotlib as mpl
from matplotlib import cm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline, FeatureUnion

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [5]:
from download_daily_data import my_list
from supervised.utils import get_numeric_categoric, delta_dataframe_with_y_columns

In [6]:
class TickersData(Dataset):
    def __init__(self, ticker_list, last_file_path, y_transform=None):
        '''
        :param ticker_list: iterable tickers
        :param last_file_path: pickle_file (e.g. _train.pickle, _test.pickle)
        '''
        self.tickers = ticker_list
        self.x = self.read_in_pickles('_x'+last_file_path)
        self.y = self.read_in_pickles('_y'+last_file_path)
        
        self._sanity_check()
        
        if y_transform is not None:
            self.y = y_transform(self.y).astype(np.float64)

    def read_in_pickles(self, last_file_path):
        numpy_tickers = []
        for ticker in self.tickers:
            with open('data/ohlc_processed/'+ticker+last_file_path, 'rb') as f:
                data = pickle.load(f)
                numpy_tickers += [data]
        numpy_tickers = np.concatenate(numpy_tickers, axis=1)
        numpy_tickers = numpy_tickers.astype(np.float64)
        return numpy_tickers
    
    def _sanity_check(self):
        assert self.x.shape[0] == self.y.shape[0], 'x.shape != y.shape: {} vs {}'.format(
                                                                    x.shape, y.shape)
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y

In [7]:
my_list = list(my_list)
pickle_files = list(map(lambda x: x.split('_')[0], 
                        os.listdir('data/ohlc_processed/')))
valid_tickers = [item for item in my_list if item in pickle_files]

In [32]:
my_dataset = TickersData(valid_tickers, '_train.pickle')

In [ ]:
class Classifier(nn.Module):
    def __init__(self, ticker_dim, data_point_dim, shift_dim, transform_dim, output_dim):
        '''
        :param ticker_dim     : dim of tickers used for the data
        :param data_point_dim : dim of a given data point (e.g. ohlc+volume = 5)
        :param shift_dim      : dim of shifts in time scales (e.g. 4 different shifts backs for returns)
        :param transform_dim  : dim of different transforms (e.g. 4 different sckit-transforms)
        :param output_dim     : dim of outputs, should be a multiple of ticker_dim (e.g. ticker_dim * 2)
        '''
        assert output_dim % ticker_dim == 0, 'output_dim should be divisible by ticker_dim'
        
        self.input_dim  = ticker_dim * shift_dim * data_point_dim * transform_dim
        self.output_dim = output_dim
        
        self.conv_channel = output_dim // ticker_dim
        
        super(Classifier, self).__init__()
        
        self.l1 = nn.Linear(self.input_dim, output_dim).double()
        
        self.c1 = nn.Conv1d(1, 
                            self.conv_channel, 
                            kernel_size=shift_dim*data_point_dim, 
                            stride=shift_dim*data_point_dim).double()
        self.c2 = nn.Conv1d(self.conv_channel, 
                            self.conv_channel, 
                            kernel_size=transform_dim,
                            stride=transform_dim).double()
        
        self.linear_repeat_dim, self.conv_repeat_dim = self._return_repeat_dim()
        
        self.c3 = nn.Conv1d(self.conv_channel, 1, (1,)).double()
        
    def _compute_conv_output_shape(self):
        temp_input = torch.ones((1, 1, self.input_dim), requires_grad=False).double()
        return self.c2(self.c1(temp_input)).shape

    def _compute_repeat_dim(self):
        temp_input = torch.ones((1, self.input_dim), requires_grad=False).double()
        linear_output_shape = self.l1(temp_input).unsqueeze(1).shape

        conv_output_shape = self._compute_conv_output_shape()
        
        return [conv / linear for conv, linear in 
                zip(conv_output_shape, linear_output_shape)]
    
    def _return_repeat_dim(self):
        conv_div_linear_dims = self._compute_repeat_dim()
        
        linear_output_repeat_dim = [item if item >=1.0 else 1.0 
                                    for item in conv_div_linear_dims]
        conv_output_repeat_dim = [1.0 if item >= 1.0 else 1/item 
                                  for item in conv_div_linear_dims]
        
        linear_output_repeat_dim, conv_output_repeat_dim = list(map(lambda x: np.array(x).astype(int), 
                                                [linear_output_repeat_dim, conv_output_repeat_dim]))
        
        return linear_output_repeat_dim, conv_output_repeat_dim
    
    def forward(self, input):

        linear = self.l1(input)
        linear = torch.sigmoid(linear).unsqueeze(1)
        linear = linear.repeat(*self.linear_repeat_dim)
        
        conv = input.unsqueeze(1)
        conv = torch.sigmoid(self.c1(conv))
        conv = torch.sigmoid(self.c2(conv))
        conv = conv.repeat(*self.conv_repeat_dim)
        
        out = linear + conv
        return torch.sigmoid(self.c3(out)).squeeze(1)
    
    def show_parameter_shapes(self):
        '''
        Use this function as a reminder of horrible param sizes...
        '''
        return [param.shape for child in self.children() for param in child.parameters()]

In [ ]:
from torch.nn.utils import clip_grad_norm_
import torch.optim as optim

model = Classifier(41, 5, 4, 4, 82)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=1e-2, weight_decay=1e-6)

In [9]:
import ignite
from ignite.metrics import BinaryAccuracy, Loss, Precision, Recall
from ignite.engine import Events, \
                          create_supervised_trainer, \
                          create_supervised_evaluator

In [10]:
import sklearn.metrics as sk_metrics
import torch.nn.functional as F

In [11]:
def zero_one(y_preds):
    return y_preds > 0.5
    
def zero_one_transform(output):
    return (zero_one(output[0])).long(), output[1].long()

In [12]:
def get_metrics(non_binary_y_target):
    metrics = {
                'accuracy':         BinaryAccuracy(output_transform=zero_one_transform),
                'bce':              Loss(nn.modules.loss.BCELoss()),
                'f1_score':         F1_Score(output_transform=zero_one_transform),
                'roc_auc':          ROC_AUC(),
                'precision':        Precision(output_transform=zero_one_transform),
                'recall':           Recall(output_transform=zero_one_transform),
                'conf_matrix':      ConfusionMatrix(output_transform=zero_one_transform),
                'positive_stat':    PositiveStatistics(non_binary_y_target),
    }
    return metrics

In [18]:
from functools import partial
import pickle

In [19]:
def binary_target(non_binary_y, threshold):
    return (non_binary_y > threshold).astype(np.int)

binary_transform = partial(binary_target, threshold=0.02)

In [20]:
train_set = TickersData(valid_tickers, '_train.pickle', y_transform=binary_transform)
test_set = TickersData(valid_tickers, '_test.pickle', y_transform=binary_transform)
train_dl = DataLoader(train_set, num_workers=1, batch_size=32)
test_dl = DataLoader(test_set, num_workers=1, batch_size=32)

In [28]:
non_binary_y_train = torch.FloatTensor(train_set.read_in_pickles('_y_train.pickle'))
non_binary_y_test  = torch.FloatTensor(test_set.read_in_pickles('_y_test.pickle'))

In [30]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [31]:
bce_loss = nn.modules.loss.BCELoss()

trainer = create_supervised_trainer(model, optimizer, bce_loss, device=device)

evaluator_train = create_supervised_evaluator(
    model, metrics=get_metrics(non_binary_y_train), device=device)
evaluator_test = create_supervised_evaluator(
    model, metrics=get_metrics(non_binary_y_test), device=device)

# evaluator = create_supervised_evaluator(
#     model,
#     metrics={
#         'accuracy'     : BinaryAccuracy(output_transform=zero_one_transform),
#         'bce'          : Loss(bce_loss),
#         'f1_score'     : F1_Score(output_transform=zero_one_transform),
#         'roc_auc'      : ROC_AUC(),
#         'precision'    : Precision(output_transform=zero_one_transform),
#         'recall'       : Recall(output_transform=zero_one_transform),
#         'conf_matrix'  : ConfusionMatrix(output_transform=zero_one_transform),
#         'positive_stat': PositiveStatistics(),
#     },
#     device=device)

In [32]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator_train.run(train_dl)
    metrics = evaluator_train.state.metrics
    print("Training Results  - Epoch: {} Avg accuracy: {:.5f}, Avg BCE: {:.5f}, F1 Score: {:.5f}, ROC_AUC: {:.5f}".format(
                  trainer.state.epoch, 
                  metrics['accuracy'], 
                  metrics['bce'],
                  metrics['f1_score'],
                  metrics['roc_auc'],))
    print("Training Results  - Epoch: {} Precision: {:.5f}, Recall: {:.5f}".format(
                  trainer.state.epoch, 
                  metrics['precision'], 
                  metrics['recall'],))
    print("Training Results  - Epoch: {} Confusion Matrix \n{}".format(
                  trainer.state.epoch, 
                  metrics['conf_matrix'],))
    print("Training Results  - Epoch: {} Pred Positive Stat: {:.5f}, {:.5f}".format(
                  trainer.state.epoch, 
                  metrics['positive_stat'][0],
                  metrics['positive_stat'][1]))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator_test.run(test_dl)
    metrics = evaluator_test.state.metrics
    print("Validation Results- Epoch: {} Avg accuracy: {:.5f}, Avg BCE: {:.5f}, F1 Score: {:.5f}, ROC_AUC: {:.5f}".format(
                  trainer.state.epoch, 
                  metrics['accuracy'], 
                  metrics['bce'],
                  metrics['f1_score'],
                  metrics['roc_auc'],))
    print("Validation Results- Epoch: {} Precision: {:.5f}, Recall: {:.5f}".format(
                  trainer.state.epoch, 
                  metrics['precision'],
                  metrics['recall'],))
    print("Validation Results- Epoch: {} Confusion Matrix: \n{}".format(
                  trainer.state.epoch, 
                  metrics['conf_matrix'],))
    print("Validation Results  - Epoch: {} Pred Positive Stat: {:.5f}, {:.5f}".format(
                  trainer.state.epoch, 
                  metrics['positive_stat'][0],
                  metrics['positive_stat'][1]))


In [35]:
trainer.run(train_dl, max_epochs=100)

Training Results  - Epoch: 1 Avg accuracy: 0.64992, Avg BCE: 0.64177, F1 Score: 0.00000, ROC_AUC: 0.55887
Training Results  - Epoch: 1 Precision: 0.00000, Recall: 0.00000
Training Results  - Epoch: 1 Confusion Matrix 
[[52547     0]
 [28305     0]]
Training Results  - Epoch: 1 Pred Positive Stat: 0.00000, -1.00000
Validation Results- Epoch: 1 Avg accuracy: 0.66348, Avg BCE: 0.63522, F1 Score: 0.00000, ROC_AUC: 0.55681
Validation Results- Epoch: 1 Precision: 0.00000, Recall: 0.00000
Validation Results- Epoch: 1 Confusion Matrix: 
[[12622     0]
 [ 6402     0]]
Validation Results  - Epoch: 1 Pred Positive Stat: 0.00000, -1.00000
Training Results  - Epoch: 2 Avg accuracy: 0.64992, Avg BCE: 0.64124, F1 Score: 0.00000, ROC_AUC: 0.56148
Training Results  - Epoch: 2 Precision: 0.00000, Recall: 0.00000
Training Results  - Epoch: 2 Confusion Matrix 
[[52547     0]
 [28305     0]]
Training Results  - Epoch: 2 Pred Positive Stat: 0.00000, -1.00000
Validation Results- Epoch: 2 Avg accuracy: 0.6634

Training Results  - Epoch: 14 Avg accuracy: 0.64999, Avg BCE: 0.63928, F1 Score: 0.00057, ROC_AUC: 0.57137
Training Results  - Epoch: 14 Precision: 0.80000, Recall: 0.00028
Training Results  - Epoch: 14 Confusion Matrix 
[[52545     2]
 [28297     8]]
Training Results  - Epoch: 14 Pred Positive Stat: 0.02219, 0.02205
Validation Results- Epoch: 14 Avg accuracy: 0.66358, Avg BCE: 0.63515, F1 Score: 0.00125, ROC_AUC: 0.55829
Validation Results- Epoch: 14 Precision: 0.66667, Recall: 0.00062
Validation Results- Epoch: 14 Confusion Matrix: 
[[12620     2]
 [ 6398     4]]
Validation Results  - Epoch: 14 Pred Positive Stat: 0.01905, 0.03542
Training Results  - Epoch: 15 Avg accuracy: 0.65003, Avg BCE: 0.63934, F1 Score: 0.00078, ROC_AUC: 0.57069
Training Results  - Epoch: 15 Precision: 0.84615, Recall: 0.00039
Training Results  - Epoch: 15 Confusion Matrix 
[[52545     2]
 [28294    11]]
Training Results  - Epoch: 15 Pred Positive Stat: 0.02230, 0.02013
Validation Results- Epoch: 15 Avg accura

Training Results  - Epoch: 27 Avg accuracy: 0.65003, Avg BCE: 0.64053, F1 Score: 0.00169, ROC_AUC: 0.56429
Training Results  - Epoch: 27 Precision: 0.61538, Recall: 0.00085
Training Results  - Epoch: 27 Confusion Matrix 
[[52532    15]
 [28281    24]]
Training Results  - Epoch: 27 Pred Positive Stat: 0.01015, 0.03709
Validation Results- Epoch: 27 Avg accuracy: 0.66343, Avg BCE: 0.63421, F1 Score: 0.00342, ROC_AUC: 0.56263
Validation Results- Epoch: 27 Precision: 0.47826, Recall: 0.00172
Validation Results- Epoch: 27 Confusion Matrix: 
[[12610    12]
 [ 6391    11]]
Validation Results  - Epoch: 27 Pred Positive Stat: 0.01132, 0.02642
Training Results  - Epoch: 28 Avg accuracy: 0.65003, Avg BCE: 0.64045, F1 Score: 0.00169, ROC_AUC: 0.56441
Training Results  - Epoch: 28 Precision: 0.61538, Recall: 0.00085
Training Results  - Epoch: 28 Confusion Matrix 
[[52532    15]
 [28281    24]]
Training Results  - Epoch: 28 Pred Positive Stat: 0.01167, 0.03845
Validation Results- Epoch: 28 Avg accura

Training Results  - Epoch: 40 Avg accuracy: 0.65008, Avg BCE: 0.64005, F1 Score: 0.00387, ROC_AUC: 0.56714
Training Results  - Epoch: 40 Precision: 0.56701, Recall: 0.00194
Training Results  - Epoch: 40 Confusion Matrix 
[[52505    42]
 [28250    55]]
Training Results  - Epoch: 40 Pred Positive Stat: 0.01212, 0.03414
Validation Results- Epoch: 40 Avg accuracy: 0.66343, Avg BCE: 0.63605, F1 Score: 0.00559, ROC_AUC: 0.55633
Validation Results- Epoch: 40 Precision: 0.48649, Recall: 0.00281
Validation Results- Epoch: 40 Confusion Matrix: 
[[12603    19]
 [ 6384    18]]
Validation Results  - Epoch: 40 Pred Positive Stat: 0.01332, 0.03497
Training Results  - Epoch: 41 Avg accuracy: 0.65004, Avg BCE: 0.64065, F1 Score: 0.00324, ROC_AUC: 0.56464
Training Results  - Epoch: 41 Precision: 0.56098, Recall: 0.00163
Training Results  - Epoch: 41 Confusion Matrix 
[[52511    36]
 [28259    46]]
Training Results  - Epoch: 41 Pred Positive Stat: 0.00941, 0.03346
Validation Results- Epoch: 41 Avg accura

Training Results  - Epoch: 53 Avg accuracy: 0.65018, Avg BCE: 0.64058, F1 Score: 0.00639, ROC_AUC: 0.56494
Training Results  - Epoch: 53 Precision: 0.56522, Recall: 0.00321
Training Results  - Epoch: 53 Confusion Matrix 
[[52477    70]
 [28214    91]]
Training Results  - Epoch: 53 Pred Positive Stat: 0.01186, 0.03274
Validation Results- Epoch: 53 Avg accuracy: 0.66400, Avg BCE: 0.63671, F1 Score: 0.01752, ROC_AUC: 0.55036
Validation Results- Epoch: 53 Precision: 0.54808, Recall: 0.00890
Validation Results- Epoch: 53 Confusion Matrix: 
[[12575    47]
 [ 6345    57]]
Validation Results  - Epoch: 53 Pred Positive Stat: 0.01285, 0.03416
Training Results  - Epoch: 54 Avg accuracy: 0.65015, Avg BCE: 0.64040, F1 Score: 0.00625, ROC_AUC: 0.56592
Training Results  - Epoch: 54 Precision: 0.55975, Recall: 0.00314
Training Results  - Epoch: 54 Confusion Matrix 
[[52477    70]
 [28216    89]]
Training Results  - Epoch: 54 Pred Positive Stat: 0.01113, 0.03344
Validation Results- Epoch: 54 Avg accura

Training Results  - Epoch: 66 Avg accuracy: 0.65014, Avg BCE: 0.63965, F1 Score: 0.00639, ROC_AUC: 0.56917
Training Results  - Epoch: 66 Precision: 0.55488, Recall: 0.00321
Training Results  - Epoch: 66 Confusion Matrix 
[[52474    73]
 [28214    91]]
Training Results  - Epoch: 66 Pred Positive Stat: 0.01143, 0.03311
Validation Results- Epoch: 66 Avg accuracy: 0.66453, Avg BCE: 0.63591, F1 Score: 0.01936, ROC_AUC: 0.55491
Validation Results- Epoch: 66 Precision: 0.59434, Recall: 0.00984
Validation Results- Epoch: 66 Confusion Matrix: 
[[12579    43]
 [ 6339    63]]
Validation Results  - Epoch: 66 Pred Positive Stat: 0.01708, 0.03387
Training Results  - Epoch: 67 Avg accuracy: 0.65010, Avg BCE: 0.63987, F1 Score: 0.00625, ROC_AUC: 0.56788
Training Results  - Epoch: 67 Precision: 0.54601, Recall: 0.00314
Training Results  - Epoch: 67 Confusion Matrix 
[[52473    74]
 [28216    89]]
Training Results  - Epoch: 67 Pred Positive Stat: 0.01163, 0.03329
Validation Results- Epoch: 67 Avg accura

Training Results  - Epoch: 79 Avg accuracy: 0.65006, Avg BCE: 0.63938, F1 Score: 0.00380, ROC_AUC: 0.56878
Training Results  - Epoch: 79 Precision: 0.56250, Recall: 0.00191
Training Results  - Epoch: 79 Confusion Matrix 
[[52505    42]
 [28251    54]]
Training Results  - Epoch: 79 Pred Positive Stat: 0.01196, 0.03294
Validation Results- Epoch: 79 Avg accuracy: 0.66385, Avg BCE: 0.63517, F1 Score: 0.00837, ROC_AUC: 0.56258
Validation Results- Epoch: 79 Precision: 0.57447, Recall: 0.00422
Validation Results- Epoch: 79 Confusion Matrix: 
[[12602    20]
 [ 6375    27]]
Validation Results  - Epoch: 79 Pred Positive Stat: 0.01947, 0.03401
Training Results  - Epoch: 80 Avg accuracy: 0.65016, Avg BCE: 0.63903, F1 Score: 0.00486, ROC_AUC: 0.57000
Training Results  - Epoch: 80 Precision: 0.58475, Recall: 0.00244
Training Results  - Epoch: 80 Confusion Matrix 
[[52498    49]
 [28236    69]]
Training Results  - Epoch: 80 Pred Positive Stat: 0.01342, 0.03121
Validation Results- Epoch: 80 Avg accura

Training Results  - Epoch: 92 Avg accuracy: 0.65036, Avg BCE: 0.64076, F1 Score: 0.00842, ROC_AUC: 0.56195
Training Results  - Epoch: 92 Precision: 0.58824, Recall: 0.00424
Training Results  - Epoch: 92 Confusion Matrix 
[[52463    84]
 [28185   120]]
Training Results  - Epoch: 92 Pred Positive Stat: 0.01257, 0.03237
Validation Results- Epoch: 92 Avg accuracy: 0.66400, Avg BCE: 0.63441, F1 Score: 0.02053, ROC_AUC: 0.56319
Validation Results- Epoch: 92 Precision: 0.54032, Recall: 0.01047
Validation Results- Epoch: 92 Confusion Matrix: 
[[12565    57]
 [ 6335    67]]
Validation Results  - Epoch: 92 Pred Positive Stat: 0.01566, 0.03538
Training Results  - Epoch: 93 Avg accuracy: 0.65045, Avg BCE: 0.64089, F1 Score: 0.00891, ROC_AUC: 0.56182
Training Results  - Epoch: 93 Precision: 0.60190, Recall: 0.00449
Training Results  - Epoch: 93 Confusion Matrix 
[[52463    84]
 [28178   127]]
Training Results  - Epoch: 93 Pred Positive Stat: 0.01312, 0.03210
Validation Results- Epoch: 93 Avg accura

In [36]:
trainer.run(train_dl, max_epochs=100)

Training Results  - Epoch: 1 Avg accuracy: 0.65035, Avg BCE: 0.64031, F1 Score: 0.00786, ROC_AUC: 0.56354
Training Results  - Epoch: 1 Precision: 0.59259, Recall: 0.00396
Training Results  - Epoch: 1 Confusion Matrix 
[[52470    77]
 [28193   112]]
Training Results  - Epoch: 1 Pred Positive Stat: 0.01243, 0.03203
Validation Results- Epoch: 1 Avg accuracy: 0.66400, Avg BCE: 0.63391, F1 Score: 0.01993, ROC_AUC: 0.56474
Validation Results- Epoch: 1 Precision: 0.54167, Recall: 0.01015
Validation Results- Epoch: 1 Confusion Matrix: 
[[12567    55]
 [ 6337    65]]
Validation Results  - Epoch: 1 Pred Positive Stat: 0.01511, 0.03504
Training Results  - Epoch: 2 Avg accuracy: 0.65040, Avg BCE: 0.63992, F1 Score: 0.00842, ROC_AUC: 0.56531
Training Results  - Epoch: 2 Precision: 0.59701, Recall: 0.00424
Training Results  - Epoch: 2 Confusion Matrix 
[[52466    81]
 [28185   120]]
Training Results  - Epoch: 2 Pred Positive Stat: 0.01273, 0.03158
Validation Results- Epoch: 2 Avg accuracy: 0.66406, 

Training Results  - Epoch: 14 Avg accuracy: 0.65037, Avg BCE: 0.64042, F1 Score: 0.00974, ROC_AUC: 0.56495
Training Results  - Epoch: 14 Precision: 0.57676, Recall: 0.00491
Training Results  - Epoch: 14 Confusion Matrix 
[[52445   102]
 [28166   139]]
Training Results  - Epoch: 14 Pred Positive Stat: 0.01292, 0.03274
Validation Results- Epoch: 14 Avg accuracy: 0.66484, Avg BCE: 0.63347, F1 Score: 0.02328, ROC_AUC: 0.56653
Validation Results- Epoch: 14 Precision: 0.60317, Recall: 0.01187
Validation Results- Epoch: 14 Confusion Matrix: 
[[12572    50]
 [ 6326    76]]
Validation Results  - Epoch: 14 Pred Positive Stat: 0.01840, 0.03414
Training Results  - Epoch: 15 Avg accuracy: 0.65044, Avg BCE: 0.64011, F1 Score: 0.01036, ROC_AUC: 0.56672
Training Results  - Epoch: 15 Precision: 0.58268, Recall: 0.00523
Training Results  - Epoch: 15 Confusion Matrix 
[[52441   106]
 [28157   148]]
Training Results  - Epoch: 15 Pred Positive Stat: 0.01260, 0.03346
Validation Results- Epoch: 15 Avg accura

Training Results  - Epoch: 27 Avg accuracy: 0.65047, Avg BCE: 0.63899, F1 Score: 0.01458, ROC_AUC: 0.57217
Training Results  - Epoch: 27 Precision: 0.56032, Recall: 0.00738
Training Results  - Epoch: 27 Confusion Matrix 
[[52383   164]
 [28096   209]]
Training Results  - Epoch: 27 Pred Positive Stat: 0.01199, 0.03270
Validation Results- Epoch: 27 Avg accuracy: 0.66369, Avg BCE: 0.63840, F1 Score: 0.02529, ROC_AUC: 0.54510
Validation Results- Epoch: 27 Precision: 0.51235, Recall: 0.01296
Validation Results- Epoch: 27 Confusion Matrix: 
[[12543    79]
 [ 6319    83]]
Validation Results  - Epoch: 27 Pred Positive Stat: 0.01175, 0.03363
Training Results  - Epoch: 28 Avg accuracy: 0.65055, Avg BCE: 0.63930, F1 Score: 0.01554, ROC_AUC: 0.57103
Training Results  - Epoch: 28 Precision: 0.56456, Recall: 0.00788
Training Results  - Epoch: 28 Confusion Matrix 
[[52375   172]
 [28082   223]]
Training Results  - Epoch: 28 Pred Positive Stat: 0.01218, 0.03229
Validation Results- Epoch: 28 Avg accura

Training Results  - Epoch: 40 Avg accuracy: 0.65050, Avg BCE: 0.64099, F1 Score: 0.01465, ROC_AUC: 0.56307
Training Results  - Epoch: 40 Precision: 0.56300, Recall: 0.00742
Training Results  - Epoch: 40 Confusion Matrix 
[[52384   163]
 [28095   210]]
Training Results  - Epoch: 40 Pred Positive Stat: 0.01231, 0.03239
Validation Results- Epoch: 40 Avg accuracy: 0.66395, Avg BCE: 0.63778, F1 Score: 0.02620, ROC_AUC: 0.54595
Validation Results- Epoch: 40 Precision: 0.52761, Recall: 0.01343
Validation Results- Epoch: 40 Confusion Matrix: 
[[12545    77]
 [ 6316    86]]
Validation Results  - Epoch: 40 Pred Positive Stat: 0.01379, 0.03406
Training Results  - Epoch: 41 Avg accuracy: 0.65056, Avg BCE: 0.64100, F1 Score: 0.01581, ROC_AUC: 0.56299
Training Results  - Epoch: 41 Precision: 0.56468, Recall: 0.00802
Training Results  - Epoch: 41 Confusion Matrix 
[[52372   175]
 [28078   227]]
Training Results  - Epoch: 41 Pred Positive Stat: 0.01318, 0.03273
Validation Results- Epoch: 41 Avg accura

Training Results  - Epoch: 53 Avg accuracy: 0.65055, Avg BCE: 0.64036, F1 Score: 0.01272, ROC_AUC: 0.56510
Training Results  - Epoch: 53 Precision: 0.58147, Recall: 0.00643
Training Results  - Epoch: 53 Confusion Matrix 
[[52416   131]
 [28123   182]]
Training Results  - Epoch: 53 Pred Positive Stat: 0.01263, 0.03218
Validation Results- Epoch: 53 Avg accuracy: 0.66516, Avg BCE: 0.63418, F1 Score: 0.02956, ROC_AUC: 0.56325
Validation Results- Epoch: 53 Precision: 0.59877, Recall: 0.01515
Validation Results- Epoch: 53 Confusion Matrix: 
[[12557    65]
 [ 6305    97]]
Validation Results  - Epoch: 53 Pred Positive Stat: 0.01831, 0.03395
Training Results  - Epoch: 54 Avg accuracy: 0.65046, Avg BCE: 0.64043, F1 Score: 0.01334, ROC_AUC: 0.56487
Training Results  - Epoch: 54 Precision: 0.56509, Recall: 0.00675
Training Results  - Epoch: 54 Confusion Matrix 
[[52400   147]
 [28114   191]]
Training Results  - Epoch: 54 Pred Positive Stat: 0.01149, 0.03215
Validation Results- Epoch: 54 Avg accura

Training Results  - Epoch: 66 Avg accuracy: 0.65070, Avg BCE: 0.63893, F1 Score: 0.01403, ROC_AUC: 0.57081
Training Results  - Epoch: 66 Precision: 0.59292, Recall: 0.00710
Training Results  - Epoch: 66 Confusion Matrix 
[[52409   138]
 [28104   201]]
Training Results  - Epoch: 66 Pred Positive Stat: 0.01245, 0.03156
Validation Results- Epoch: 66 Avg accuracy: 0.66532, Avg BCE: 0.63325, F1 Score: 0.03046, ROC_AUC: 0.57077
Validation Results- Epoch: 66 Precision: 0.60606, Recall: 0.01562
Validation Results- Epoch: 66 Confusion Matrix: 
[[12557    65]
 [ 6302   100]]
Validation Results  - Epoch: 66 Pred Positive Stat: 0.01801, 0.03438
Training Results  - Epoch: 67 Avg accuracy: 0.65070, Avg BCE: 0.63902, F1 Score: 0.01548, ROC_AUC: 0.57071
Training Results  - Epoch: 67 Precision: 0.58268, Recall: 0.00784
Training Results  - Epoch: 67 Confusion Matrix 
[[52388   159]
 [28083   222]]
Training Results  - Epoch: 67 Pred Positive Stat: 0.01242, 0.03084
Validation Results- Epoch: 67 Avg accura

Training Results  - Epoch: 79 Avg accuracy: 0.65086, Avg BCE: 0.63946, F1 Score: 0.02136, ROC_AUC: 0.57074
Training Results  - Epoch: 79 Precision: 0.57037, Recall: 0.01088
Training Results  - Epoch: 79 Confusion Matrix 
[[52315   232]
 [27997   308]]
Training Results  - Epoch: 79 Pred Positive Stat: 0.01410, 0.03274
Validation Results- Epoch: 79 Avg accuracy: 0.66484, Avg BCE: 0.63305, F1 Score: 0.04149, ROC_AUC: 0.56877
Validation Results- Epoch: 79 Precision: 0.55200, Recall: 0.02156
Validation Results- Epoch: 79 Confusion Matrix: 
[[12510   112]
 [ 6264   138]]
Validation Results  - Epoch: 79 Pred Positive Stat: 0.01658, 0.03456
Training Results  - Epoch: 80 Avg accuracy: 0.65072, Avg BCE: 0.63860, F1 Score: 0.02128, ROC_AUC: 0.57407
Training Results  - Epoch: 80 Precision: 0.55920, Recall: 0.01085
Training Results  - Epoch: 80 Confusion Matrix 
[[52305   242]
 [27998   307]]
Training Results  - Epoch: 80 Pred Positive Stat: 0.01345, 0.03230
Validation Results- Epoch: 80 Avg accura

Training Results  - Epoch: 92 Avg accuracy: 0.65039, Avg BCE: 0.63953, F1 Score: 0.01765, ROC_AUC: 0.56993
Training Results  - Epoch: 92 Precision: 0.54043, Recall: 0.00897
Training Results  - Epoch: 92 Confusion Matrix 
[[52331   216]
 [28051   254]]
Training Results  - Epoch: 92 Pred Positive Stat: 0.01166, 0.03124
Validation Results- Epoch: 92 Avg accuracy: 0.66532, Avg BCE: 0.63309, F1 Score: 0.03778, ROC_AUC: 0.56942
Validation Results- Epoch: 92 Precision: 0.58140, Recall: 0.01953
Validation Results- Epoch: 92 Confusion Matrix: 
[[12532    90]
 [ 6277   125]]
Validation Results  - Epoch: 92 Pred Positive Stat: 0.01734, 0.03304
Training Results  - Epoch: 93 Avg accuracy: 0.65055, Avg BCE: 0.64045, F1 Score: 0.02025, ROC_AUC: 0.56609
Training Results  - Epoch: 93 Precision: 0.54784, Recall: 0.01032
Training Results  - Epoch: 93 Confusion Matrix 
[[52306   241]
 [28013   292]]
Training Results  - Epoch: 93 Pred Positive Stat: 0.01295, 0.03172
Validation Results- Epoch: 93 Avg accura

In [37]:
trainer.run(train_dl, max_epochs=100)

Training Results  - Epoch: 1 Avg accuracy: 0.65099, Avg BCE: 0.63739, F1 Score: 0.02346, ROC_AUC: 0.57858
Training Results  - Epoch: 1 Precision: 0.57360, Recall: 0.01198
Training Results  - Epoch: 1 Confusion Matrix 
[[52295   252]
 [27966   339]]
Training Results  - Epoch: 1 Pred Positive Stat: 0.01321, 0.03204
Validation Results- Epoch: 1 Avg accuracy: 0.66411, Avg BCE: 0.63531, F1 Score: 0.04456, ROC_AUC: 0.56113
Validation Results- Epoch: 1 Precision: 0.52098, Recall: 0.02327
Validation Results- Epoch: 1 Confusion Matrix: 
[[12485   137]
 [ 6253   149]]
Validation Results  - Epoch: 1 Pred Positive Stat: 0.01442, 0.03370
Training Results  - Epoch: 2 Avg accuracy: 0.65102, Avg BCE: 0.63728, F1 Score: 0.02272, ROC_AUC: 0.57891
Training Results  - Epoch: 2 Precision: 0.57848, Recall: 0.01159
Training Results  - Epoch: 2 Confusion Matrix 
[[52308   239]
 [27977   328]]
Training Results  - Epoch: 2 Pred Positive Stat: 0.01350, 0.03217
Validation Results- Epoch: 2 Avg accuracy: 0.66395, 

Training Results  - Epoch: 14 Avg accuracy: 0.65093, Avg BCE: 0.63799, F1 Score: 0.02508, ROC_AUC: 0.57491
Training Results  - Epoch: 14 Precision: 0.56366, Recall: 0.01282
Training Results  - Epoch: 14 Confusion Matrix 
[[52266   281]
 [27942   363]]
Training Results  - Epoch: 14 Pred Positive Stat: 0.01277, 0.03117
Validation Results- Epoch: 14 Avg accuracy: 0.66453, Avg BCE: 0.63658, F1 Score: 0.04831, ROC_AUC: 0.55917
Validation Results- Epoch: 14 Precision: 0.53289, Recall: 0.02530
Validation Results- Epoch: 14 Confusion Matrix: 
[[12480   142]
 [ 6240   162]]
Validation Results  - Epoch: 14 Pred Positive Stat: 0.01352, 0.03387
Training Results  - Epoch: 15 Avg accuracy: 0.65095, Avg BCE: 0.63806, F1 Score: 0.02596, ROC_AUC: 0.57476
Training Results  - Epoch: 15 Precision: 0.56287, Recall: 0.01328
Training Results  - Epoch: 15 Confusion Matrix 
[[52255   292]
 [27929   376]]
Training Results  - Epoch: 15 Pred Positive Stat: 0.01270, 0.03144
Validation Results- Epoch: 15 Avg accura

Training Results  - Epoch: 27 Avg accuracy: 0.65095, Avg BCE: 0.63799, F1 Score: 0.02373, ROC_AUC: 0.57498
Training Results  - Epoch: 27 Precision: 0.56977, Recall: 0.01212
Training Results  - Epoch: 27 Confusion Matrix 
[[52288   259]
 [27962   343]]
Training Results  - Epoch: 27 Pred Positive Stat: 0.01306, 0.03166
Validation Results- Epoch: 27 Avg accuracy: 0.66421, Avg BCE: 0.63735, F1 Score: 0.04371, ROC_AUC: 0.55949
Validation Results- Epoch: 27 Precision: 0.52518, Recall: 0.02281
Validation Results- Epoch: 27 Confusion Matrix: 
[[12490   132]
 [ 6256   146]]
Validation Results  - Epoch: 27 Pred Positive Stat: 0.01320, 0.03309
Training Results  - Epoch: 28 Avg accuracy: 0.65097, Avg BCE: 0.63872, F1 Score: 0.02508, ROC_AUC: 0.57144
Training Results  - Epoch: 28 Precision: 0.56630, Recall: 0.01282
Training Results  - Epoch: 28 Confusion Matrix 
[[52269   278]
 [27942   363]]
Training Results  - Epoch: 28 Pred Positive Stat: 0.01295, 0.03097
Validation Results- Epoch: 28 Avg accura

Training Results  - Epoch: 40 Avg accuracy: 0.65088, Avg BCE: 0.63806, F1 Score: 0.02386, ROC_AUC: 0.57463
Training Results  - Epoch: 40 Precision: 0.56373, Recall: 0.01219
Training Results  - Epoch: 40 Confusion Matrix 
[[52280   267]
 [27960   345]]
Training Results  - Epoch: 40 Pred Positive Stat: 0.01301, 0.03141
Validation Results- Epoch: 40 Avg accuracy: 0.66505, Avg BCE: 0.63385, F1 Score: 0.04554, ROC_AUC: 0.56973
Validation Results- Epoch: 40 Precision: 0.55474, Recall: 0.02374
Validation Results- Epoch: 40 Confusion Matrix: 
[[12500   122]
 [ 6250   152]]
Validation Results  - Epoch: 40 Pred Positive Stat: 0.01624, 0.03435
Training Results  - Epoch: 41 Avg accuracy: 0.65081, Avg BCE: 0.63887, F1 Score: 0.02386, ROC_AUC: 0.57180
Training Results  - Epoch: 41 Precision: 0.55825, Recall: 0.01219
Training Results  - Epoch: 41 Confusion Matrix 
[[52274   273]
 [27960   345]]
Training Results  - Epoch: 41 Pred Positive Stat: 0.01283, 0.03135
Validation Results- Epoch: 41 Avg accura

Training Results  - Epoch: 53 Avg accuracy: 0.65086, Avg BCE: 0.64030, F1 Score: 0.02393, ROC_AUC: 0.56598
Training Results  - Epoch: 53 Precision: 0.56169, Recall: 0.01222
Training Results  - Epoch: 53 Confusion Matrix 
[[52277   270]
 [27959   346]]
Training Results  - Epoch: 53 Pred Positive Stat: 0.01313, 0.03152
Validation Results- Epoch: 53 Avg accuracy: 0.66463, Avg BCE: 0.63465, F1 Score: 0.04748, ROC_AUC: 0.56555
Validation Results- Epoch: 53 Precision: 0.53716, Recall: 0.02484
Validation Results- Epoch: 53 Confusion Matrix: 
[[12485   137]
 [ 6243   159]]
Validation Results  - Epoch: 53 Pred Positive Stat: 0.01489, 0.03374
Training Results  - Epoch: 54 Avg accuracy: 0.65088, Avg BCE: 0.63961, F1 Score: 0.02460, ROC_AUC: 0.56864
Training Results  - Epoch: 54 Precision: 0.56151, Recall: 0.01258
Training Results  - Epoch: 54 Confusion Matrix 
[[52269   278]
 [27949   356]]
Training Results  - Epoch: 54 Pred Positive Stat: 0.01334, 0.03162
Validation Results- Epoch: 54 Avg accura

Training Results  - Epoch: 66 Avg accuracy: 0.65087, Avg BCE: 0.63865, F1 Score: 0.02575, ROC_AUC: 0.57204
Training Results  - Epoch: 66 Precision: 0.55755, Recall: 0.01318
Training Results  - Epoch: 66 Confusion Matrix 
[[52251   296]
 [27932   373]]
Training Results  - Epoch: 66 Pred Positive Stat: 0.01327, 0.03265
Validation Results- Epoch: 66 Avg accuracy: 0.66479, Avg BCE: 0.63506, F1 Score: 0.04807, ROC_AUC: 0.56330
Validation Results- Epoch: 66 Precision: 0.54209, Recall: 0.02515
Validation Results- Epoch: 66 Confusion Matrix: 
[[12486   136]
 [ 6241   161]]
Validation Results  - Epoch: 66 Pred Positive Stat: 0.01499, 0.03381
Training Results  - Epoch: 67 Avg accuracy: 0.65093, Avg BCE: 0.63844, F1 Score: 0.02447, ROC_AUC: 0.57247
Training Results  - Epoch: 67 Precision: 0.56550, Recall: 0.01251
Training Results  - Epoch: 67 Confusion Matrix 
[[52275   272]
 [27951   354]]
Training Results  - Epoch: 67 Pred Positive Stat: 0.01342, 0.03169
Validation Results- Epoch: 67 Avg accura

Training Results  - Epoch: 79 Avg accuracy: 0.65081, Avg BCE: 0.63858, F1 Score: 0.03182, ROC_AUC: 0.57416
Training Results  - Epoch: 79 Precision: 0.54206, Recall: 0.01639
Training Results  - Epoch: 79 Confusion Matrix 
[[52155   392]
 [27841   464]]
Training Results  - Epoch: 79 Pred Positive Stat: 0.01228, 0.03219
Validation Results- Epoch: 79 Avg accuracy: 0.66474, Avg BCE: 0.63686, F1 Score: 0.05762, ROC_AUC: 0.55463
Validation Results- Epoch: 79 Precision: 0.53279, Recall: 0.03046
Validation Results- Epoch: 79 Confusion Matrix: 
[[12451   171]
 [ 6207   195]]
Validation Results  - Epoch: 79 Pred Positive Stat: 0.01244, 0.03265
Training Results  - Epoch: 80 Avg accuracy: 0.65097, Avg BCE: 0.63828, F1 Score: 0.03064, ROC_AUC: 0.57526
Training Results  - Epoch: 80 Precision: 0.55266, Recall: 0.01576
Training Results  - Epoch: 80 Confusion Matrix 
[[52186   361]
 [27859   446]]
Training Results  - Epoch: 80 Pred Positive Stat: 0.01284, 0.03217
Validation Results- Epoch: 80 Avg accura

Training Results  - Epoch: 92 Avg accuracy: 0.65099, Avg BCE: 0.63828, F1 Score: 0.03084, ROC_AUC: 0.57619
Training Results  - Epoch: 92 Precision: 0.55364, Recall: 0.01586
Training Results  - Epoch: 92 Confusion Matrix 
[[52185   362]
 [27856   449]]
Training Results  - Epoch: 92 Pred Positive Stat: 0.01338, 0.03340
Validation Results- Epoch: 92 Avg accuracy: 0.66463, Avg BCE: 0.63742, F1 Score: 0.05649, ROC_AUC: 0.55406
Validation Results- Epoch: 92 Precision: 0.53056, Recall: 0.02983
Validation Results- Epoch: 92 Confusion Matrix: 
[[12453   169]
 [ 6211   191]]
Validation Results  - Epoch: 92 Pred Positive Stat: 0.01320, 0.03429
Training Results  - Epoch: 93 Avg accuracy: 0.65114, Avg BCE: 0.63826, F1 Score: 0.03019, ROC_AUC: 0.57700
Training Results  - Epoch: 93 Precision: 0.56354, Recall: 0.01551
Training Results  - Epoch: 93 Confusion Matrix 
[[52207   340]
 [27866   439]]
Training Results  - Epoch: 93 Pred Positive Stat: 0.01349, 0.03369
Validation Results- Epoch: 93 Avg accura

In [38]:
trainer.run(train_dl, max_epochs=100)

Training Results  - Epoch: 1 Avg accuracy: 0.65105, Avg BCE: 0.63721, F1 Score: 0.02704, ROC_AUC: 0.57965
Training Results  - Epoch: 1 Precision: 0.56647, Recall: 0.01385
Training Results  - Epoch: 1 Confusion Matrix 
[[52247   300]
 [27913   392]]
Training Results  - Epoch: 1 Pred Positive Stat: 0.01390, 0.03314
Validation Results- Epoch: 1 Avg accuracy: 0.66479, Avg BCE: 0.63531, F1 Score: 0.05147, ROC_AUC: 0.56600
Validation Results- Epoch: 1 Precision: 0.53894, Recall: 0.02702
Validation Results- Epoch: 1 Confusion Matrix: 
[[12474   148]
 [ 6229   173]]
Validation Results  - Epoch: 1 Pred Positive Stat: 0.01365, 0.03351
Training Results  - Epoch: 2 Avg accuracy: 0.65098, Avg BCE: 0.63753, F1 Score: 0.02804, ROC_AUC: 0.57870
Training Results  - Epoch: 2 Precision: 0.55907, Recall: 0.01438
Training Results  - Epoch: 2 Confusion Matrix 
[[52226   321]
 [27898   407]]
Training Results  - Epoch: 2 Pred Positive Stat: 0.01342, 0.03318
Validation Results- Epoch: 2 Avg accuracy: 0.66432, 

Training Results  - Epoch: 14 Avg accuracy: 0.65113, Avg BCE: 0.63723, F1 Score: 0.02992, ROC_AUC: 0.57997
Training Results  - Epoch: 14 Precision: 0.56347, Recall: 0.01537
Training Results  - Epoch: 14 Confusion Matrix 
[[52210   337]
 [27870   435]]
Training Results  - Epoch: 14 Pred Positive Stat: 0.01348, 0.03439
Validation Results- Epoch: 14 Avg accuracy: 0.66416, Avg BCE: 0.63602, F1 Score: 0.05390, ROC_AUC: 0.56279
Validation Results- Epoch: 14 Precision: 0.51852, Recall: 0.02843
Validation Results- Epoch: 14 Confusion Matrix: 
[[12453   169]
 [ 6220   182]]
Validation Results  - Epoch: 14 Pred Positive Stat: 0.01286, 0.03376
Training Results  - Epoch: 15 Avg accuracy: 0.65107, Avg BCE: 0.63711, F1 Score: 0.03111, ROC_AUC: 0.57971
Training Results  - Epoch: 15 Precision: 0.55720, Recall: 0.01600
Training Results  - Epoch: 15 Confusion Matrix 
[[52187   360]
 [27852   453]]
Training Results  - Epoch: 15 Pred Positive Stat: 0.01297, 0.03462
Validation Results- Epoch: 15 Avg accura

Training Results  - Epoch: 27 Avg accuracy: 0.65103, Avg BCE: 0.63819, F1 Score: 0.02542, ROC_AUC: 0.57631
Training Results  - Epoch: 27 Precision: 0.56966, Recall: 0.01300
Training Results  - Epoch: 27 Confusion Matrix 
[[52269   278]
 [27937   368]]
Training Results  - Epoch: 27 Pred Positive Stat: 0.01343, 0.03243
Validation Results- Epoch: 27 Avg accuracy: 0.66479, Avg BCE: 0.63425, F1 Score: 0.04721, ROC_AUC: 0.56890
Validation Results- Epoch: 27 Precision: 0.54296, Recall: 0.02468
Validation Results- Epoch: 27 Confusion Matrix: 
[[12489   133]
 [ 6244   158]]
Validation Results  - Epoch: 27 Pred Positive Stat: 0.01355, 0.03298
Training Results  - Epoch: 28 Avg accuracy: 0.65097, Avg BCE: 0.63823, F1 Score: 0.02549, ROC_AUC: 0.57541
Training Results  - Epoch: 28 Precision: 0.56508, Recall: 0.01304
Training Results  - Epoch: 28 Confusion Matrix 
[[52263   284]
 [27936   369]]
Training Results  - Epoch: 28 Pred Positive Stat: 0.01322, 0.03273
Validation Results- Epoch: 28 Avg accura

Training Results  - Epoch: 40 Avg accuracy: 0.65087, Avg BCE: 0.63984, F1 Score: 0.02796, ROC_AUC: 0.56999
Training Results  - Epoch: 40 Precision: 0.55238, Recall: 0.01434
Training Results  - Epoch: 40 Confusion Matrix 
[[52218   329]
 [27899   406]]
Training Results  - Epoch: 40 Pred Positive Stat: 0.01276, 0.03329
Validation Results- Epoch: 40 Avg accuracy: 0.66537, Avg BCE: 0.63274, F1 Score: 0.04758, ROC_AUC: 0.57064
Validation Results- Epoch: 40 Precision: 0.56383, Recall: 0.02484
Validation Results- Epoch: 40 Confusion Matrix: 
[[12499   123]
 [ 6243   159]]
Validation Results  - Epoch: 40 Pred Positive Stat: 0.01386, 0.03385
Training Results  - Epoch: 41 Avg accuracy: 0.65088, Avg BCE: 0.63961, F1 Score: 0.02783, ROC_AUC: 0.57097
Training Results  - Epoch: 41 Precision: 0.55342, Recall: 0.01427
Training Results  - Epoch: 41 Confusion Matrix 
[[52221   326]
 [27901   404]]
Training Results  - Epoch: 41 Pred Positive Stat: 0.01288, 0.03330
Validation Results- Epoch: 41 Avg accura

Training Results  - Epoch: 53 Avg accuracy: 0.65070, Avg BCE: 0.63884, F1 Score: 0.02614, ROC_AUC: 0.57333
Training Results  - Epoch: 53 Precision: 0.54532, Recall: 0.01339
Training Results  - Epoch: 53 Confusion Matrix 
[[52231   316]
 [27926   379]]
Training Results  - Epoch: 53 Pred Positive Stat: 0.01226, 0.03252
Validation Results- Epoch: 53 Avg accuracy: 0.66379, Avg BCE: 0.63676, F1 Score: 0.04623, ROC_AUC: 0.55501
Validation Results- Epoch: 53 Precision: 0.50987, Recall: 0.02421
Validation Results- Epoch: 53 Confusion Matrix: 
[[12473   149]
 [ 6247   155]]
Validation Results  - Epoch: 53 Pred Positive Stat: 0.01249, 0.03472
Training Results  - Epoch: 54 Avg accuracy: 0.65070, Avg BCE: 0.63849, F1 Score: 0.02607, ROC_AUC: 0.57486
Training Results  - Epoch: 54 Precision: 0.54545, Recall: 0.01335
Training Results  - Epoch: 54 Confusion Matrix 
[[52232   315]
 [27927   378]]
Training Results  - Epoch: 54 Pred Positive Stat: 0.01221, 0.03282
Validation Results- Epoch: 54 Avg accura

Training Results  - Epoch: 66 Avg accuracy: 0.65100, Avg BCE: 0.63795, F1 Score: 0.03078, ROC_AUC: 0.57651
Training Results  - Epoch: 66 Precision: 0.55446, Recall: 0.01583
Training Results  - Epoch: 66 Confusion Matrix 
[[52187   360]
 [27857   448]]
Training Results  - Epoch: 66 Pred Positive Stat: 0.01260, 0.03388
Validation Results- Epoch: 66 Avg accuracy: 0.66406, Avg BCE: 0.63667, F1 Score: 0.05389, ROC_AUC: 0.55760
Validation Results- Epoch: 66 Precision: 0.51558, Recall: 0.02843
Validation Results- Epoch: 66 Confusion Matrix: 
[[12451   171]
 [ 6220   182]]
Validation Results  - Epoch: 66 Pred Positive Stat: 0.01346, 0.03461
Training Results  - Epoch: 67 Avg accuracy: 0.65095, Avg BCE: 0.63743, F1 Score: 0.03064, ROC_AUC: 0.57877
Training Results  - Epoch: 67 Precision: 0.55198, Recall: 0.01576
Training Results  - Epoch: 67 Confusion Matrix 
[[52185   362]
 [27859   446]]
Training Results  - Epoch: 67 Pred Positive Stat: 0.01246, 0.03404
Validation Results- Epoch: 67 Avg accura

Training Results  - Epoch: 79 Avg accuracy: 0.65113, Avg BCE: 0.63823, F1 Score: 0.03145, ROC_AUC: 0.57552
Training Results  - Epoch: 79 Precision: 0.55990, Recall: 0.01618
Training Results  - Epoch: 79 Confusion Matrix 
[[52187   360]
 [27847   458]]
Training Results  - Epoch: 79 Pred Positive Stat: 0.01313, 0.03458
Validation Results- Epoch: 79 Avg accuracy: 0.66474, Avg BCE: 0.63667, F1 Score: 0.05818, ROC_AUC: 0.55831
Validation Results- Epoch: 79 Precision: 0.53243, Recall: 0.03077
Validation Results- Epoch: 79 Confusion Matrix: 
[[12449   173]
 [ 6205   197]]
Validation Results  - Epoch: 79 Pred Positive Stat: 0.01484, 0.03497
Training Results  - Epoch: 80 Avg accuracy: 0.65113, Avg BCE: 0.63786, F1 Score: 0.03311, ROC_AUC: 0.57648
Training Results  - Epoch: 80 Precision: 0.55645, Recall: 0.01706
Training Results  - Epoch: 80 Confusion Matrix 
[[52162   385]
 [27822   483]]
Training Results  - Epoch: 80 Pred Positive Stat: 0.01296, 0.03382
Validation Results- Epoch: 80 Avg accura

Training Results  - Epoch: 92 Avg accuracy: 0.65093, Avg BCE: 0.63870, F1 Score: 0.03230, ROC_AUC: 0.57407
Training Results  - Epoch: 92 Precision: 0.54767, Recall: 0.01664
Training Results  - Epoch: 92 Confusion Matrix 
[[52158   389]
 [27834   471]]
Training Results  - Epoch: 92 Pred Positive Stat: 0.01292, 0.03366
Validation Results- Epoch: 92 Avg accuracy: 0.66364, Avg BCE: 0.63639, F1 Score: 0.05242, ROC_AUC: 0.55634
Validation Results- Epoch: 92 Precision: 0.50427, Recall: 0.02765
Validation Results- Epoch: 92 Confusion Matrix: 
[[12448   174]
 [ 6225   177]]
Validation Results  - Epoch: 92 Pred Positive Stat: 0.01121, 0.03367
Training Results  - Epoch: 93 Avg accuracy: 0.65107, Avg BCE: 0.63857, F1 Score: 0.03357, ROC_AUC: 0.57458
Training Results  - Epoch: 93 Precision: 0.55242, Recall: 0.01731
Training Results  - Epoch: 93 Confusion Matrix 
[[52150   397]
 [27815   490]]
Training Results  - Epoch: 93 Pred Positive Stat: 0.01305, 0.03343
Validation Results- Epoch: 93 Avg accura